In [1]:
%matplotlib notebook

# system
from src.system_creator import SystemCreator

# Grid
from src.utils import Grid, pos_in_grid, convert_to_grid_datatype

# Particles
from src.utils import Particle

# injection 
from src.utils import inject

# advection
from src.utils import advect
from src.utils import euler_explicit, leap_frog

# collisions
from src.utils import handler_wall_collision, handler_wall_collision_point, make_collisions_vectorized, make_collisions_out_walls, deal_with_corner

# utils 
from src.utils import gaussian, maxwellian_flux, maxwellian_mean_speed, get_mass_part, mean_free_path, mean_free_time

# systems
from src.utils import thruster

# plotting 
from src.plotting import plot_boundaries, plot_particles, plot_grid, plot_system

# collisions between particles
from src.utils import handler_particles_collisions # candidates, index_choosen_couples, probability, is_colliding, reflect, 

# other imports
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd

# os path module
from os import path

np.random.seed(1111)

In [2]:
# TODO : make dictionnary
iterations = 10000

# System :
dz = 0.001
idx_out_walls = [0,10] # 2nd and 3rd walls are out walls. - thruster

    # Thruster
dp = 0.001
segments = thruster(w_in = 5*dp, l_in = 3*dp, w1 = 3*dp, l1 = dp, l_int = dp, w2 = dp, l2 = 5*dp, w_out = 5*dp, l_out = dp, offsets = np.array([0,0]))
system = SystemCreator(segments)

offsets = system.get_offsets()
system_shape = system.system_shape()
a = system.get_dir_vects()
segments = system.get_segments()

# grid :
mean_number_per_cell = 1000
max_number_per_cell = 10*mean_number_per_cell
resolutions = np.array((5,11), dtype = int) # thruster

grid = Grid(resolutions, max_number_per_cell)
volume_cell = dz * system_shape[0]/resolutions[0] * system_shape[1]/resolutions[1]

# Particles - 1 type 
density = 3.2e19 # m-3
n_simu = mean_number_per_cell*np.sum(resolutions) # number of particles in the simulated system
n_real = volume_cell * density * np.sum(resolutions) # number of particles in the real system
mr = n_real/n_simu # macro particules ratio = number of particles in the real system / number of macro part in the simulated system
density_dsmc = density/mr
temperature = 300 # K

part_type = 'I'
charge, mass, radius = 0, get_mass_part(53, 53, 74), 2e-10
size_array = 2*max_number_per_cell*np.sum(resolutions)
v_mean = maxwellian_mean_speed(temperature, mass)
container = Particle(part_type, charge, mass, radius, size_array)
cross_section = container.get_params()[3]

# mean free path and time
mfp = mean_free_path(cross_section, density)
typical_lenght = 0.001
mft = mean_free_time(typical_lenght, v_mean = v_mean)

    # Injection params
in_wall = np.array([0,0,0.005,0.0], dtype = float) # thruster
in_vect = np.array([0,-1], dtype = float)

debit = maxwellian_flux(density_dsmc, v_mean)*np.linalg.norm(in_wall[:2]-in_wall[2:])*dz
vel_std = gaussian(temperature, mass)
dt = 5e-7 # in sec, should be a fraction of the mean free time

# advection
def f(arr, dt):
    return np.zeros(shape = (arr.shape[0], 3))
args = []
scheme = euler_explicit

In [3]:
# NAME tests
name_test = '1005-thruster'

In [4]:
name_df = '{}.h5'.format(name_test)
name_coll = '{}-collisions.npy'.format(name_test)

if(path.exists('{}.h5'.format(name_test))):
    os.system('rm -f -r {}'.format(name_df))
    os.system('rm -f -r {}'.format(name_coll))
    
store = pd.HDFStore(name_df)

df = pd.DataFrame(columns = ['x','y','vx','vy','vz'])

remains = 0
saving_period = 1
# dsmc saving
averages = np.full(shape = grid.current.shape, fill_value = mean_number_per_cell)
remains_per_cell = np.zeros(shape = grid.current.shape, dtype = float)
# pmax = np.full(shape = grid.current.shape, fill_value = 2*vel_std*cross_section)
pmax = 2*vel_std*cross_section
arr_nb_colls = np.zeros((iterations, resolutions[0], resolutions[1]))

print('|{:^10}|{:^10}|{:^10}|{:^10}|{:^10}|'.format(' it ', ' INIT ', ' INJECT ', ' DEL ', ' TRY'))
print('{:-^56}'.format(''))

for it in range(iterations): # tqdm
    n1 = container.get_current()
                   
    # injecting particles
    new, remains = inject(in_wall, in_vect, debit, vel_std, radius, dt, remains)
    container.add_multiple(new)
                   
    n2 = container.get_current()-n1
    
    # PHASE : ADVECTING
        # MOVING PARTICLES
    arr = container.get_particles()
    
    if(it%saving_period==0):
        df = df.append(pd.DataFrame(data=arr, index=[it]*arr.shape[0], columns = ['x','y','vx','vy','vz']))
    
    advect(arr, f, dt, args, scheme) # advect is inplace
    
        # HANDLING BOUNDARIES 
    count = np.full(fill_value = True, shape = arr.shape[0])
    idxes_out = []
    c = 0
    while(np.sum(count, where = count == True) > 0):
        c+=1
        ct, cp = handler_wall_collision_point(arr[count], segments, a) # handler_wall_collision(arr[count], segments, a, radius)
        count, idxes_out_ = make_collisions_out_walls(arr, a, ct, cp, idx_out_walls, count) # idxes_out : indexes of the particles (in arr) that got out of the system
        idxes_out.append(idxes_out_)

    idxes_out = np.concatenate(idxes_out)
    container.delete_multiple(idxes_out)
    arr = container.get_particles()
    
    # PHASE : COLLISIONS
        # UPDATING GRID - HARD RESET
    grid.reset()
    positions = pos_in_grid(arr[:,:2], resolutions, offsets, system_shape)
    particles = convert_to_grid_datatype(positions, new = positions.shape[0])
    grid.add_multiple(particles)

        # DSMC
        # TODO: make parallel
    currents = grid.get_currents()
    remains_per_cell, nb_colls = handler_particles_collisions([arr], grid.get_grid(), currents, dt, averages, pmax, cross_section, volume_cell, mr, remains_per_cell)
    arr_nb_colls[it,:,:] = nb_colls
    
    # PLOTTING AND SAVING (OPTIONAL)
    if(it%100==0):
        store.append('df', df)
        df = pd.DataFrame(columns = ['x','y','vx','vy','vz']) # this way not too much memory is used
        print('{:^10}|{:^10}|{:^10}|{:^10}|{:^10}'.format(it, n1, n2, idxes_out.shape[0], c))
   
# saving back
np.save(file = name_coll, arr = arr_nb_colls, allow_pickle=False)
store.append('df', df)

# plotting system - last frame 
fig, ax = plot_system(arr, segments, radius, resolutions, system_shape, offsets)
fig.show()

|    it    |   INIT   |  INJECT  |   DEL    |    TRY   |
--------------------------------------------------------
    0     |    0     |    13    |    0     |    1     
   100    |   1192   |    14    |    7     |    3     
   200    |   1646   |    14    |    11    |    2     
   300    |   1904   |    14    |    12    |    2     
   400    |   2027   |    14    |    6     |    3     
   500    |   2080   |    14    |    18    |    2     
   600    |   2144   |    14    |    17    |    3     
   700    |   2194   |    14    |    10    |    2     
   800    |   2246   |    14    |    16    |    2     
   900    |   2261   |    14    |    8     |    3     
